# Spherical forward model for extended EEG sources

In [1]:
import math
import numpy as np
import scipy
from scipy import special
from scipy.linalg import solve
from IPython.display import clear_output

#### Position in terms of (radius, theta, phi)

In [2]:
def raio(x):
    
    r = math.sqrt(x[0]**2 + x[1]**2 + x[2]**2)
    
    return(r)

########################################################################################################

def theta_ang(x):
    
    r = raio(x)
    theta = math.acos(x[2]/r)
    
    return(theta)

########################################################################################################

def phi_ang(x):    
    
    phi = math.atan2(x[1],x[0])
        
    return(phi)

#### Dipole Rotation

In [3]:
def alpha_const(x):
    
    alpha = math.atan2(x[1],x[2])
    
    return(alpha)

########################################################################################################

def beta_const(x):
    
    x = rot_x_mom(x)
    beta = math.atan2(x[0],x[2])
    
    return(beta)

########################################################################################################

def gamma_const(x,alpha,beta):
    
    x = rot_x_pos(x,alpha)
    x = rot_y_pos(x,beta)
    gamma = math.atan2(-x[1],x[0])
     
    return(gamma)

########################################################################################################

def rot_x_pos(pos,alpha):
    
    pos = (pos[0], pos[1]*math.cos(alpha) - pos[2]*math.sin(alpha), pos[1]*math.sin(alpha) + pos[2]*math.cos(alpha))
    
    return(pos)

########################################################################################################

def rot_x_mom(mom):
    
    mom = (mom[0], 0, math.sqrt(mom[1]**2 + mom[2]**2))
    
    return(mom)

########################################################################################################

def rot_y_pos(pos,beta):
    
    pos = (pos[0]*math.cos(beta) - pos[2]*math.sin(beta), pos[1], pos[0]*math.sin(beta) + pos[2]*math.cos(beta))
    
    return(pos)

########################################################################################################

def rot_y_mom(mom):
    
    mom = (0, mom[1], math.sqrt(mom[0]**2 + mom[2]**2))
    
    return(mom)

########################################################################################################

def rot_z_pos(pos,gamma):
    
    pos = (pos[0]*math.cos(gamma) - pos[1]*math.sin(gamma), pos[0]*math.sin(gamma) + pos[1]*math.cos(gamma), pos[2])
    
    return(pos)

########################################################################################################

def new_Position(pos,alpha,beta,gamma):
    
    pos = rot_x_pos(pos,alpha)
    pos = rot_y_pos(pos,beta)
    pos = rot_z_pos(pos,gamma)
    
    return(pos)

########################################################################################################

def new_Moment(mom):
    
    mom = rot_y_mom(rot_x_mom(mom))
    
    return(mom)

#### MSup & MInf calculation

In [4]:
def Msup_lm(M,sigma,b,theta,max_l,m,l,r):
    
    if l == 0:
        Msup = 0
    else:
        if (m == 0):
            delta = 1
        else:
            delta = 0
            
        ALP = scipy.special.lpmn(max_l, max_l, math.cos(theta))[0][m,l-1] * (-1)**m
        Msup = (M / (4*math.pi*sigma*(b**2))) * (2 - delta) * (math.factorial(l-m) / math.factorial(l+m-1)) * (b/r)**(l+1) * ALP
    
    return(Msup)
    
########################################################################################################

def Minf_lm(M,sigma,b,theta,max_l,m,l,r):
    
    if l >= max_l:
        Minf = 0
    else:
        if (m == 0):
            delta = 1
        else:
            delta = 0
            
        ALP = scipy.special.lpmn(max_l, max_l, math.cos(theta))[0][m,l+1] * (-1)**m
        Minf = (M / (4*math.pi*sigma*(b**2))) * (2 - delta) * (m-l-1) * (math.factorial(l-m) / math.factorial(l+m)) * (b/r)**(-l) * ALP
        
    return(Minf)

#### Clm & Dlm calculation (Unknown variables )

In [5]:
def Clm_Dlm_Vector(r_eeg,rayon,conductivity,M,b,theta,max_l,m,l,dip_layer,r_layer,cond_layer):
    
    n = len(rayon)

    #Matrix A
    A1 = np.zeros((n-1,n))

    for i in range(n-1):
        for j in range(n):
            if i == j:
                A1[i,j] = (rayon[i]/r_eeg)**l
            elif j == i + 1:
                A1[i,j] = -(rayon[i]/r_eeg)**l

            
    A2 = np.zeros((n,n))

    for i in range(n):
        for j in range(n):
            if i == j:
                if i == n:
                    A2[i,j] = l * (rayon[i]/r_eeg)**l #(l-1)
                else:
                    A2[i,j] = (conductivity[i]/rayon[i]) * l * (rayon[i]/r_eeg)**l #(l-1)
            elif j == i + 1:
                A2[i,j] = -(conductivity[i+1]/rayon[i]) * l * (rayon[i]/r_eeg)**l #(l-1)

            
    A3 = np.zeros((n-1,n-1))        

    for i in range(n-1):
        for j in range(n-1):
            if i == j:
                A3[i,j] = -1 / (rayon[i]/r_eeg)**(l+1)
            elif i == j + 1:
                A3[i,j] = 1 / (rayon[i]/r_eeg)**(l+1)
                

    A4 = np.zeros((n,n-1))
            
    for i in range(n):
        for j in range(n-1):
            if i == j:
                A4[i,j] = ((conductivity[i+1]/rayon[i])*(l+1)) / ((rayon[i]/r_eeg)*((rayon[i]/r_eeg)**l))
            elif i == j + 1:
                if i == n:
                    A4[i,j] = -(l+1) / ((rayon[i]/r_eeg)*((rayon[i]/r_eeg)**l))
                else:
                    A4[i,j] = -((conductivity[i]/rayon[i])*(l+1)) / ((rayon[i]/r_eeg)*((rayon[i]/r_eeg)**l))
                    
    
    A = np.zeros((2*n - 1, 2*n - 1))

    for i in range(2*n - 1):
        for j in range(2*n - 1):
            if j < n:
                if i < n-1:
                    A[i,j] = A1[i,j]
                else:
                    A[i,j] = A2[i-(n-1),j]
            else:
                if i < n-1:
                    A[i,j] = A3[i,j-n]
                else:
                    A[i,j] = A4[i-(n-1),j-n]   
                    
    
    #Matrix B
    B = np.zeros((2*n - 1,1))  
    B[dip_layer,0] = -Msup_lm(M,cond_layer,b,theta,max_l,m,l,r_layer)
    B[dip_layer+n-1,0] = cond_layer * (l+1) * Msup_lm(M,cond_layer,b,theta,max_l,m,l,r_layer)/r_layer
    
    
    #Matrix X
    if l == 0:
        X = np.zeros((2*n - 1,1))
    else:
        X = solve(A,B)
    
    return(X)

#### Analytical Model

In [6]:
def V_0b(max_l,r_eeg,phi_eeg,theta_eeg,rayon,conductivity,M,b,theta,dip_layer,r_layer,cond_layer):
    
    total_sum = 0
    
    for i in range(max_l+1):
        for j in range(i+1):
            Clm_Dlm = Clm_Dlm_Vector(r_eeg,rayon,conductivity,M,b,theta,max_l,j,i,dip_layer,r_layer,cond_layer)
            ALP = scipy.special.lpmn(max_l, max_l, math.cos(theta_eeg))[0][j,i] * (-1)**j
            coeff = (Clm_Dlm[0] + Minf_lm(M,cond_layer,b,theta,max_l,j,i,r_eeg))
            potential = coeff * math.cos(j*phi_eeg) * ALP #* r_eeg**i
            
            total_sum += potential 
              
    return(total_sum)

########################################################################################################

def V_br(max_l,r_eeg,phi_eeg,theta_eeg,rayon,conductivity,M,b,theta,dip_layer,r_layer,cond_layer):
    
    total_sum = 0
    
    for i in range(max_l+1):
        for j in range(i+1):
            Clm_Dlm = Clm_Dlm_Vector(r_eeg,rayon,conductivity,M,b,theta,max_l,j,i,dip_layer,r_layer,cond_layer)
            ALP = scipy.special.lpmn(max_l, max_l, math.cos(theta_eeg))[0][j,i] * (-1)**j
            coeff = (Clm_Dlm[0]) + (Msup_lm(M,cond_layer,b,theta,max_l,j,i,r_eeg))
            potential = coeff * math.cos(j*phi_eeg) * ALP
            
            total_sum += potential
            
    return(total_sum)

########################################################################################################

def V_rr(max_l,r_eeg,phi_eeg,theta_eeg,rayon,conductivity,M,b,theta,eeg_layer,dip_layer,r_layer,cond_layer):
    
    total_sum = 0
    
    for i in range(max_l+1):
        for j in range(i+1):
            Clm_Dlm = Clm_Dlm_Vector(r_eeg,rayon,conductivity,M,b,theta,max_l,j,i,dip_layer,r_layer,cond_layer)
            ALP = scipy.special.lpmn(max_l, max_l, math.cos(theta_eeg))[0][j,i] * (-1)**j
            coeff = ((Clm_Dlm[eeg_layer] * r_eeg**i) + (Clm_Dlm[eeg_layer+(len(rayon)-1)] * r_eeg**-(i+1)))
            potential = coeff * math.cos(j*phi_eeg) * ALP
                
            total_sum += potential
            
    return(total_sum)

#### Spherical foward model (potential calculation)

In [7]:
def V_Potential(position,moment,rayon,conductivity,max_l,eeg):
    
    #rotation const
    alpha = alpha_const(moment)
    beta = beta_const(moment)
    gamma = gamma_const(position,alpha,beta)
    
    #dipole
    p = new_Position(position,alpha,beta,gamma)
    q = new_Moment(moment)
    
    b = raio(p)
    theta = theta_ang(p)
    phi = phi_ang(p)
    M = raio(q)
    
    #eeg
    eeg = new_Position(eeg,alpha,beta,gamma)
    
    r_eeg = round(raio(eeg),5)
    theta_eeg = theta_ang(eeg)
    phi_eeg = phi_ang(eeg)
    
    #layer
    dip_layer = 0
    r_layer = rayon[0]
    cond_layer = conductivity[0]
    for i in range(len(rayon)-1):
        if (b >= rayon[i]):
            dip_layer = i+1
            r_layer = rayon[i+1]
            cond_layer = conductivity[i+1]
        
    
    #potential
    if (r_eeg >= 0):
        if (r_eeg < b):
            potential = V_0b(max_l,r_eeg,phi_eeg,theta_eeg,rayon,conductivity,M,b,theta,dip_layer,r_layer,cond_layer)
            
        elif (r_eeg <= rayon[0]):
            potential = V_br(max_l,r_eeg,phi_eeg,theta_eeg,rayon,conductivity,M,b,theta,dip_layer,r_layer,cond_layer)
        
        else:
            for i in range(1,len(rayon)):
                if (r_eeg <= rayon[i]):
                    potential = V_rr(max_l,r_eeg,phi_eeg,theta_eeg,rayon,conductivity,M,b,theta,i,dip_layer,r_layer,cond_layer)
                    r_eeg = rayon[len(rayon)-1] + 100
                    
    
    return(float(potential))

#### Spherical foward model for extendend sources (potential calculation)

In [8]:
def V_Potential_Tetrahedron(tetra,moment,rayon,conductivity,max_l,eeg):
    n = 0
    total_sum = 0
    
    for lambda_1 in np.linspace(0,1,5):
        for lambda_2 in np.linspace(0,1-lambda_1,5):
            for lambda_3 in np.linspace(0,1-lambda_1-lambda_2,5):
                lambda_4 = 1 - lambda_1 - lambda_2 - lambda_3
                            
                position = np.array(tetra[0])*lambda_1 + np.array(tetra[1])*lambda_2 + np.array(tetra[2])*lambda_3 + np.array(tetra[3])*lambda_4
                moment_tetra = np.array(moment[0])*lambda_1 + np.array(moment[1])*lambda_2 + np.array(moment[2])*lambda_3 + np.array(moment[3])*lambda_4
                    
                potential = V_Potential(position,moment_tetra,rayon,cond,max_l,eeg)
                    
                total_sum += potential
                n += 1
                      
    return(total_sum/n)

#### Examples

<img src="Rayon_Cond_Ex.png"/>

In [9]:
max_l = 100

#dipole
position = [0,0,0.1]
moment = [1,1,1]

#head parameters (inside to outside)
rayon = [0.87,0.92,1]
cond = [1, 0.0125, 1]

#eeg position
eeg = [0,0,1]


#Spherical foward model (potential calculation)
V_Potential(position,moment,rayon,cond,max_l,eeg)


0.17564676084412037

In [10]:
max_l = 100

#tetrahedron
tetra = [[0.01,0.01,0.11],[0.01,-0.01,0.09],[-0.01,0.01,0.09],[-0.01,-0.01,0.11]] # 0.01 around point (0,0,0.1)
moment = [[1,1,1],[1,1,1],[1,1,1],[1,1,1]]

#head parameters (inside to outside)
rayon = [0.87,0.92,1]
cond = [1, 0.0125, 1]

#eeg position
eeg = [0,0,1]

#Spherical foward model for extendend sources (potential calculation)
V_Potential_Tetrahedron(tetra,moment,rayon,cond,max_l,eeg)


0.1753876781870304